In [ ]:
import psycopg2
import sqlite3
import json
import os

%load_ext nb_black

In [ ]:
def connect():
    conn = psycopg2.connect(
        user="postgres",
        password=os.environ.get("POSTGRES_PASS", ""),
        host="localhost",
        port=5432,
        database="venmo",
    )
    return conn

In [ ]:
conn = connect()
lite_con = sqlite3.connect("F:/data-venmo/venmo.db")

In [ ]:
cur = conn.cursor()
cur.execute("SELECT table_name, column_name, data_type FROM information_schema.columns")
tables = cur.fetchall()

table_cols = {"users": {}, "transactions": {}, "user_to_transactions": {}}
for table, col, dtype in tables:
    if table not in table_cols:
        continue
    table_cols[table][col] = dtype

lite_cur = lite_con.cursor()
for table, col_types in table_cols.items():
    lite_cur.execute(
        """
    CREATE TABLE {}
        ({})
    """.format(
            table,
            ",".join([col + " " + col_type for col, col_type in col_types.items()]),
        )
    )
lite_con.commit()

In [ ]:
table_cols

In [ ]:
for table, col_types in table_cols.items():
    print("Exporting", table)
    with conn.cursor(name="read_all_" + table) as cursor:
        cursor.itersize = 10_000
        cursor.execute(
            """
        SELECT {} FROM {}
        """.format(
                ", ".join(col for col in col_types), table
            )
        )
        batch = []
        for i, item in enumerate(cursor):
            item = [(json.dumps(v) if isinstance(v, dict) else v) for v in item]
            batch.append(item)
            if len(batch) == 10_000:
                print(" saving...", i)
                lite_cur.executemany(
                    "INSERT INTO {} VALUES ({})".format(
                        table, ",".join(["?" for _ in col_types])
                    ),
                    batch,
                )
                lite_con.commit()
                batch = []
        if len(batch) > 0:
            lite_cur.executemany(
                "INSERT INTO {} VALUES ({})".format(
                    table, ",".join(["?" for _ in col_types])
                ),
                batch,
            )
            lite_con.commit()